In [1]:
import os

In [2]:
%pwd

'd:\\Git\\DeepLearning_Classification\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Git\\DeepLearning_Classification'

In [5]:
#https://dagshub.com/delphineasir/DeepLearning_Classification.mlflow
!pip install dagshub

In [6]:
import dagshub
dagshub.init(repo_owner='delphineasir', repo_name='DeepLearning_Classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "delphineasir/DeepLearning_Classification"

Repository delphineasir/DeepLearning_Classification initialized!

RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [7]:
import dagshub
import mlflow

# Initialize DagsHub
dagshub.init(repo_owner='delphineasir', repo_name='DeepLearning_Classification', mlflow=True)

# Set the experiment name
experiment_name = "DeepLearning_Classification_Experiment"

# Check if the experiment exists and create it if it does not
if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(experiment_name)

# Set the experiment to log runs to
mlflow.set_experiment(experiment_name)

# Start an MLflow run
with mlflow.start_run():
    mlflow.log_param('parameter_name', 'value')
    mlflow.log_metric('metric_name', 1)


Initialized MLflow to track repo "delphineasir/DeepLearning_Classification"

Repository delphineasir/DeepLearning_Classification initialized!

In [ ]:
"""os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="entbappy"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6824692c47a369aa6f9eac5b10041d5c8edbcef0"
"""

In [9]:
import tensorflow as tf
import os
os.chdir("d:\\Git\\DeepLearning_Classification")


In [10]:
%pwd

'd:\\Git\\DeepLearning_Classification'

In [11]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [13]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT_SCAN",                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
            mlflow_uri="https://dagshub.com/delphineasir/DeepLearning_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [16]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-06-27 11:53:47,693: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-06-27 11:53:47,708: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-27 11:53:47,709: INFO: common: created directory at: artifacts]
Found 2207 images belonging to 2 classes.
138/138 [==============================] - 386s 3s/step - loss: 4.3064 - accuracy: 0.6964
[2024-06-27 12:00:15,174: INFO: common: json file saved at: scores.json]


2024/06/27 12:00:16 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-06-27 12:00:20,071: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpfg7b1nbt\model\data\model\assets
[2024-06-27 12:00:21,634: INFO: builder_impl: Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpfg7b1nbt\model\data\model\assets]


c:\Users\ASUS\anaconda3\envs\KIDNEY\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/06/27 12:01:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
